In [ ]:
print("hello")

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("languagedetection.csv")

In [ ]:
df.head(10)

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df

In [ ]:
print(df.isnull().sum())

In [ ]:
df = df.dropna()
df = df.drop_duplicates()

In [ ]:
df['Text'] = df['Text'].str.strip()

In [ ]:
df.head(10)

In [ ]:
%pip install regex
import regex as re 

def cleaning(text):
    if not isinstance(text, str):
        return ""
    # Keep letters (from all languages) and spaces only
    text = re.sub(r"[^\p{L}\s]", "", text)   # remove non-letter, non-space chars
    text = re.sub(r"\s+", " ", text).strip() # normalize spaces
    return text

df["Text"] = df['Text'].apply(cleaning)

In [ ]:
df

In [ ]:
%pip install scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df

In [ ]:
v = TfidfVectorizer()
X = v.fit_transform(df['Text'])
y = df['Language']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# nb = MultinomialNB()
# nb.fit(X_train, y_train)
# y_pred = nb.predict(X_test)
# acc = accuracy_score(y_test, y_pred)
# acc

In [ ]:
# lr = LogisticRegression()
# lr.fit(X_train, y_train)
# y_pred = lr.predict(X_test)
# acc = accuracy_score(y_test, y_pred)
# acc

In [ ]:
# SVM = LinearSVC()
# SVM.fit(X_train, y_train)
# y_pred = SVM.predict(X_test)
# acc = accuracy_score(y_test, y_pred)
# acc

In [ ]:
# rf = RandomForestClassifier()
# rf.fit(X_train, y_train)
# y_pred = rf.predict(X_test)
# acc = accuracy_score(y_test, y_pred)
# acc

In [ ]:
models = {
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42),
    "Naive Bayes": MultinomialNB(),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "SVM": LinearSVC()
}

results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc
    print(f"{name} Accuracy: {acc:.4f}")

In [ ]:
%pip install joblib
import joblib
best_model = max(results, key=results.get)
joblib.dump(models[best_model], f"{best_model.replace(' ', '_')}_model.pkl")
print(f"✅ Best model '{best_model}' saved!")

In [ ]:
# best_model_name = max(results, key=results.get)
# joblib.dump(v, 'language_vectorizer.pkl')
# best_model_object = models[best_model_name]
# print(f"\n🏆 The best model is '{best_model_name}' with an accuracy of {results[best_model_name]:.4f}")

# # Save the best performing model with a consistent filename
# joblib.dump(best_model_object, 'best_language_model.pkl')
# print(f"✅ Best model saved as 'best_language_model.pkl'")

In [ ]:
# print("\n\n--- Starting Language Prediction ---")

# # Load the saved vectorizer and the best model
# loaded_vectorizer = joblib.load('language_vectorizer.pkl')
# loaded_model = joblib.load('best_language_model.pkl')
# print("✅ Model and vectorizer loaded successfully!")

# # Get input from the user
# user_input = input("\nPlease enter a sentence to identify its language: ")

# # Prepare the input for the model
# # 1. Put the input into a list
# # 2. Transform it using the loaded vectorizer
# input_vec = loaded_vectorizer.transform([user_input])

# # Make a prediction
# prediction = loaded_model.predict(input_vec)

# # Get the single prediction result from the array
# predicted_language = prediction[0]

# # Display the final result
# print(f"\nInput Text: '{user_input}'")
# print(f"Predicted Language: {predicted_language}")

In [ ]:
import joblib

# Save both model and vectorizer
joblib.dump(models[best_model], f"{best_model}_model.pkl")
joblib.dump(v, "tfidf_vectorizer.pkl")
print(f"✅ Best model and vectorizer saved!")


# Load the trained best model
best_model_loaded = joblib.load("SVM_model.pkl")  # Replace 'SVM_model.pkl' with your actual best model filename

# Load the TF-IDF vectorizer
vectorizer_loaded = joblib.load("tfidf_vectorizer.pkl")

In [ ]:
import regex as re  # multilingual-safe cleaning

def clean_input(text):
    if not isinstance(text, str):
        return ""
    # Keep letters (from all languages) and spaces only
    text = re.sub(r"[^\p{L}\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [ ]:
while True:
    user_text = input("Enter text to detect language (type 1 to stop): ")

    if user_text.strip() == "1":
        print("\nExiting... 👋")
        break
        
    cleaned_text = clean_input(user_text)
    X_input = vectorizer_loaded.transform([cleaned_text])
    predicted_language = best_model_loaded.predict(X_input)[0]

    print(f"\nInput Text: '{user_text}'")
    print(f"Predicted Language: {predicted_language}")